In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import requests
import json
from lostark_api_token import Token
 
headers = {
    'accept' : 'application/json',
    'authorization' : Token
}

In [3]:
# 찾아야 할 아이템 딕셔너리
tier_3_item_dict = {
    "찬란한 명예의 돌파석": "66110224",
    "명예의 파편 주머니(소)": "66130131",
    "명예의 파편 주머니(중)": "66130132",
    "명예의 파편 주머니(대)": "66130133",
    "정제된 파괴강석": "66102005",
    "정제된 수호강석": "66102105",
    "태양의 은총": "66111121",
    "태양의 축복": "66111122",
    "태양의 가호": "66111123",
    "야금술_몽환[13-15]": "66112531",
    "재봉술_몽환[13-15]": "66112534",
    "야금술_몽환[16-19]": "66112532",
    "재봉술_몽환[16-19]": "66112535",
    "최상급 오레하 융화 재료": "6861011",
    "오레하 유물": "6885708",
    "오레하 태양 잉어": "6885608",
    "오레하 두툼한 생고기": "6885508"
}

tier_4_item_dict = {
    "운명의 돌파석":"66110225",
    "운명의 파편 주머니(소)": "66130141",
    "운명의 파편 주머니(중)": "66130142",
    "운명의 파괴석": "66102006",
    "운명의 수호석": "66102106",
    "용암의 숨결": "66111131",
    "빙하의 숨결": "66111132",
    "야금술_업화": "66112543",
    "재봉술_업화": "66112546",
    "아비도스 융화 재료": "6861012",
    "아비도스 두툼한 생고기": "6885509",
    "아비도스 철광석": "6884408",
    "아비도스 목재": "6884308",
    "아비도스 유물": "6885709",
    "아비도스 태양 잉어": "6885609",
    "아비도스 들꽃": "6882109",
    "장인의 재봉술 : 1단계":"66112712",
    "장인의 재봉술 : 2단계":"66112714",
    "장인의 야금술 : 1단계":"66112711",
    "장인의 야금술 : 2단계":"66112713"
}

life_gradient_dict = {
    "묵직한 철광석": "6882404",
    "철광석": "6882401",
    "부드러운 목재": "6882304",
    "목재": "6882301",
    "들꽃": "6882101",
    "수줍은 들꽃": "6882104",
    "두툼한 생고기": "6882501",
    "다듬은 생고기": "6882504",
    "생선": "6882601",
    "붉은 살 생선": "6882604",
    "고대 유물": "6882701",
    "희귀한 유물": "6882704"
}

In [4]:
tier_3_item_df = pd.DataFrame(tier_3_item_dict.items(), columns=['item_name', 'item_code'])
tier_3_item_df['tier'] = 3
tier_4_item_df = pd.DataFrame(tier_4_item_dict.items(), columns=['item_name', 'item_code'])
tier_4_item_df['tier'] = 4
life_gradient_df = pd.DataFrame(life_gradient_dict.items(), columns=['item_name', 'item_code'])
life_gradient_df['tier'] = 1
total_item_df = pd.concat([tier_3_item_df, tier_4_item_df, life_gradient_df], ignore_index=True)

In [5]:
total_item_df

,item_name,item_code,tier
0,찬란한 명예의 돌파석,66110224,3
1,명예의 파편 주머니(소),66130131,3
2,명예의 파편 주머니(중),66130132,3
3,명예의 파편 주머니(대),66130133,3
4,정제된 파괴강석,66102005,3
5,정제된 수호강석,66102105,3
6,태양의 은총,66111121,3
7,태양의 축복,66111122,3
8,태양의 가호,66111123,3
9,야금술_몽환[13-15],66112531,3


In [6]:
# api 호출 테스트
url = 'https://developer-lostark.game.onstove.com/markets/items/66110224'
response = requests.get(url, headers=headers)
jsonObject = response.json()
print(response)
print(jsonObject)

# json 출력 코드
for list in jsonObject :
    print(list.get("Stats"))

<Response [200]>
[{'Name': '찬란한 명예의 돌파석', 'TradeRemainCount': None, 'BundleCount': 1, 'Stats': [{'Date': '2025-02-28', 'AvgPrice': 33.5, 'TradeCount': 274366}, {'Date': '2025-02-27', 'AvgPrice': 31.1, 'TradeCount': 950665}, {'Date': '2025-02-26', 'AvgPrice': 28.9, 'TradeCount': 909807}, {'Date': '2025-02-25', 'AvgPrice': 28.0, 'TradeCount': 727296}, {'Date': '2025-02-24', 'AvgPrice': 29.9, 'TradeCount': 795840}, {'Date': '2025-02-23', 'AvgPrice': 30.6, 'TradeCount': 1065736}, {'Date': '2025-02-22', 'AvgPrice': 29.1, 'TradeCount': 1060671}, {'Date': '2025-02-21', 'AvgPrice': 29.0, 'TradeCount': 827074}, {'Date': '2025-02-20', 'AvgPrice': 29.0, 'TradeCount': 883551}, {'Date': '2025-02-19', 'AvgPrice': 29.1, 'TradeCount': 944089}, {'Date': '2025-02-18', 'AvgPrice': 28.1, 'TradeCount': 919209}, {'Date': '2025-02-17', 'AvgPrice': 28.6, 'TradeCount': 918666}, {'Date': '2025-02-16', 'AvgPrice': 29.1, 'TradeCount': 1197565}, {'Date': '2025-02-15', 'AvgPrice': 29.0, 'TradeCount': 1168858}], 'To

In [7]:
# 아이템 코드로 아이템 정보 가져오기
def get_item_info(item_code):
    url = f'https://developer-lostark.game.onstove.com/markets/items/{item_code}'
    response = requests.get(url, headers=headers)
    jsonObject = response.json()
    return jsonObject

# 전 날의 아이템 정보를 가져오기
def item_info_to_df(item_info):
    name = item_info[0]['Name']
    stats = item_info[0]['Stats']
    item_info_df = pd.DataFrame(stats)
    item_info_df['item_name'] = name
    return item_info_df


In [8]:
# 아이템 코드로 아이템 정보를 불러와 데이터프레임으로 만들고 아이템 이름으로 csv 파일 만들기
for item_code in total_item_df['item_code']:
    item_info = get_item_info(item_code)
    item_info_df = item_info_to_df(item_info)
    item_info_df = item_info_df.iloc[1:].reset_index(drop=True)
    item_info_df = item_info_df.sort_values(by='Date', ascending=True)
    item_name = item_info_df['item_name'][0]
    
    # 특수 문자 제거 및 띄어쓰기 제거
    safe_item_name = re.sub(r'[\\/:*?"<>|[\]]', '', item_name)  # 특수 문자를 제거
    safe_item_name = safe_item_name.replace(' ', '')  # 띄어쓰기 제거
    
    # 파일 저장
    item_info_df.to_csv(f'./data/{safe_item_name}.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 1월 22일 등장하기 시작한 운명의 파편 주머니(대) 파일 생성
item_info = get_item_info(66130143)
item_info_df = item_info_to_df(item_info)
item_info_df = item_info_df.iloc[1:].reset_index(drop=True)
item_info_df = item_info_df.sort_values(by='Date', ascending=True)
item_name = item_info_df['item_name'][0]
# 특수 문자 제거 및 띄어쓰기 제거
safe_item_name = re.sub(r'[\\/:*?"<>|[\]]', '', item_name)  # 특수 문자를 제거
safe_item_name = safe_item_name.replace(' ', '')  # 띄어쓰기 제거

# 파일 저장
item_info_df.to_csv(f'./data/{safe_item_name}.csv', index=False, encoding='utf-8-sig')

In [8]:
# 새로 등장한 상급 재련 보조 아이템 파일 생성
new_items = [66112711, 66112712, 66112713, 66112714]

for i in new_items:
    item_info = get_item_info(i)
    item_info_df = item_info_to_df(item_info)
    item_info_df = item_info_df.iloc[1:].reset_index(drop=True)
    item_info_df = item_info_df.sort_values(by='Date', ascending=True)
    item_name = item_info_df['item_name'][0]
    # 특수 문자 제거 및 띄어쓰기 제거
    safe_item_name = re.sub(r'[\\/:*?"<>|[\]]', '', item_name)  # 특수 문자를 제거
    safe_item_name = safe_item_name.replace(' ', '')  # 띄어쓰기 제거

    # 파일 저장
    item_info_df.to_csv(f'./data/{safe_item_name}.csv', index=False, encoding='utf-8-sig')
